# 🧠 LLM Playground

A lightweight environment for experimenting with:

- PDF chunking  
- Embeddings via Ollama  
- pgvector similarity search  
- A minimal RAG pipeline  

**Init Environment:**
 - Start Environment: .\venv\Scripts\activate
 - Stop Environment: deactivate
 - See [readme.md](../README.md) for Environment setup 

## 1 - Initalize Project
Initialize the core components:

- PostgreSQL + pgvector database  
- Local LLM client (Ollama)  
- RAG engine (chunking, embedding, search, answering)  

In [6]:
import sys
import traceback
from pathlib import Path

# Add app directory to path for imports
sys.path.insert(0, str(Path.cwd()))

print("✓ Loading core modules...")
from core.database import Database
from core.rag import RAG
from core.ollama_client import OllamaClient

# Init DB
db = Database()
db.init_schema()
db_embeddings = db.get_embedding_dimension()
print(f"✓ Database ready: Embedding - Dimensions", db_embeddings)

# Init Olama
ollama_client = OllamaClient()
result = ollama_client.generate("Hello from the OOP client!")
print("✓ Ollama Response: ", result)

# Init RAG
try:
    rag = RAG()
    print("✓ LLM Client: ", rag.llm)
except Exception as e:
    print(f"⚠ RAG initialization warning: {e}")
    traceback.print_exc()
    rag = None

print("✓ System running...")

✓ Loading core modules...
✓ Database ready: Embedding - Dimensions 956
✓ Ollama Response:  Hello from the OOP client! How can I help you today?
✓ LLM Client:  <core.ollama_client.OllamaClient object at 0x000001EDBAFE5C50>
✓ System running...


### 1.1 - RAG: Chunk PDF File

Convert a PDF into semantic text chunks and store embeddings in the vector database.

In [10]:
# PDF chunking & Embeddings
try:
    chunks = rag.chunk_pdf("./store/resume.pdf")
    print(f"✓ Processed PDF into {len(chunks)} chunks")
except Exception as e:
    print("⚠ PDF processing error:", e)


✓ Processed PDF into 6 chunks


## 2 - RAG: Search Query

Perform a semantic vector search using pgvector to retrieve the most relevant chunks.

### 2.1 - Re‑Initialize RAG (Optional)

Useful when reloading the notebook or after modifying RAG logic.


In [11]:
from core.rag import RAG

try:
    rag = RAG()
    print("✓ LLM Client: ", rag.llm)
except Exception as e:
    print(f"⚠ RAG initialization warning: {e}")
    traceback.print_exc()
    rag = None

✓ LLM Client:  <core.ollama_client.OllamaClient object at 0x000001EDBC5A17C0>


### 2.2 - Embedded Vector Search 

Run a similarity search using the query embedding against stored chunk embeddings.

In [12]:
## Search Query
search_query = "im looking for a web-full-stack developer"
result = []

try:
    results = rag.search(search_query, limit=3)
    print("✓ Search results:")
    for id, content, distance in results:
        print("\n---")
        print("Chunk ID:", id)
        print("Distance:", distance)
        print(content)
except Exception as e:
    print("⚠ Search error:", e)

✓ Search results:

---
Chunk ID: 17
Distance: 64.11421312173105
, MySQL / PSQL)
• SPA - Applikationsentwicklung: Frontend (Vue3), Backend (Laravel), Datenbanken
• SaaS, Webapplikationen, Cross-Plattform-Development
• IT, Prozesse und Schnittstellen
• Daten- & Informationsmanagement
• VS Code, Docker, GitHub, Hosting (Plesk)
• SEO, GDPR, Google Analytics, Google Tag Manager
August 2014 – September 2018 B.Sc. Betriebsökonomie, BFH (Bern)
01.03.2017 – 31.08.2017 Austauschsemester, Daejeon (Südkorea)
• Management in Practice: „Strategie“
• Führung und Organisationen
• Marketing und Kommunikation
• Accounting, Finance, Taxt, Recht, VWL, Methodik, Statistik
Mai 2014 – August 2014 Sprachaufenthalt, Kalifornien (USA)
• Cambridge English Level 1 Certificate (B2)
Ausbildung:
01.08.2010 – 31.07.2011 Technische Berufsmaturität, Gibb
01.08.2007 – 31.07.2011 Elektro

---
Chunk ID: 13
Distance: 55.851035216597595
KOMPETENZEN
Carlson Patrick, 33 Hardskills: Softskills:
• Requirements Engineering • Ler

### 2.3 - LLM Answer (RAG)

Generate a relevance‑aware answer using the best‑fit chunk from the vector search.

In [13]:
try:
    answer = rag.answer(search_query)
    print("\n✓ RAG Answer:\n", answer)
except Exception as e:
    print("⚠ Agent error:", e)


✓ RAG Answer:
 Here's an example of how you can use the context to identify relevant information and explain why it fits:

**Context:** A web-full-stack developer is looking for a new project.

**Question:** What are some things that a web-full-stack developer should know?

**Answer:** A web-full-stack developer should be familiar with the following concepts:

* Frontend development (client-side) and backend development (server-side)
* RESTful APIs and HTTP requests
* HTML, CSS, JavaScript, and other frontend technologies
* Databases like MySQL, PostgreSQL, MongoDB, and Firebase
* Version control systems like Git
* Versioning tools like SVN or GitHub
* Testing frameworks like Jest, Mocha, or Postman
* Security best practices for web applications

**Relevant Evidence:**

* "A web-full-stack developer should be familiar with the latest versions of popular frontend libraries and frameworks."
* "They should know how to use a version control system like Git effectively."
* "They should und